In [1]:
import os
import cv2
from tqdm import tqdm
from scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import jaccard_score
from collections import defaultdict

In [2]:
from calculate_iou import cluster_pixel_values, generate_mask, generate_one_dim_mask, find_central_point, calculate_iou

In [8]:
pred_img = cv2.imread('../output_bbox_gcn/g2p_layout_haroon_split_3stages_2023_02_22_17_30_32/Image_eval/count_000000001_real_floor_plan.png')
print(pred_img.shape)
pred_img[0,0]

(260, 260, 3)


array([ 84, 139,  84], dtype=uint8)

In [9]:
pixels = []
for i in range(12110):
    img_id = i + 1
    img_id = format(img_id, '09d')
    gt_img_path = 'count_' + img_id + '_real_floor_plan.png'
    pred_img_path = 'count_' + img_id + '_fake_floor_plan.png'
    gt_img = cv2.imread('../output_bbox_gcn/g2p_layout_haroon_split_3stages_2023_02_22_17_30_32/Image_eval/' + gt_img_path)
    pred_img = cv2.imread('../output_bbox_gcn/g2p_layout_haroon_split_3stages_2023_02_22_17_30_32/Image_eval/' + pred_img_path)
    for j in range(260):
        for k in range(260):
            pixels.append(pred_img[j,k])

In [10]:
len(pixels)

818636000

In [13]:
str(list(pixels[0]))

'[84, 139, 84]'

In [14]:
pixel_dict = dict.fromkeys([str(list(pix)) for pix in pixels], 0)

In [15]:
pixel_dict.keys()

dict_keys(['[84, 139, 84]', '[255, 255, 255]', '[0, 0, 0]', '[255, 0, 255]', '[115, 198, 205]', '[139, 26, 85]', '[130, 134, 139]', '[128, 0, 0]', '[255, 144, 30]', '[58, 58, 139]', '[235, 206, 135]', '[42, 42, 165]', '[0, 100, 0]'])

In [3]:
# define rooms types and their corresponding colors
last_5k_type = ['84, 139, 84','255, 255, 255', '0, 0, 0', '255, 0, 255', '115, 198, 205', '139, 26, 85', '130, 134, 139', '128, 0, 0', '255, 144, 30', '58, 58, 139', '235, 206, 135', '42, 42, 165', '0, 100, 0']
# last_5k_type = ['255,255,255', '0,0,0', '0,0,255', '170,232,238', '128,128,240', '230,216,173', '0,215,255', '0,165,255', '35,142,107', '221,160,221', '0,255,255', '214,112,218']
last_dict = dict.fromkeys(last_5k_type)
for key in last_dict.keys():
    last_dict[key] = [int(val) for val in key.split(',')]

In [5]:
# calculate IoU
from tqdm import tqdm
Macro_IoUs = []
Micro_IoUs = []
for i in tqdm(range(12110)):
    img_id = i + 1
    img_id = format(img_id, '09d')
    gt_img_path = 'count_' + img_id + '_real_floor_plan.png'
    pred_img_path = 'count_' + img_id + '_fake_floor_plan.png'
    gt_image = cv2.imread('../output_bbox_gcn/g2p_layout_haroon_split_3stages_2023_02_22_17_30_32/Image_eval/' + gt_img_path)
    pred_image = cv2.imread('../output_bbox_gcn/g2p_layout_haroon_split_3stages_2023_02_22_17_30_32/Image_eval/' + pred_img_path)

    # new_image = cluster_pixel_values(pred_image, last_dict)

    dict_pred_mask = generate_mask(pred_image, last_dict)
    dict_gt_mask = generate_mask(gt_image, last_dict)

    one_dim_pred_mask = generate_one_dim_mask(dict_pred_mask)
    one_dim_gt_mask = generate_one_dim_mask(dict_gt_mask)

    rooms = ['84, 139, 84','255, 255, 255', '0, 0, 0', '255, 0, 255', '115, 198, 205', '139, 26, 85', '130, 134, 139', '128, 0, 0', '255, 144, 30', '58, 58, 139', '235, 206, 135', '42, 42, 165', '0, 100, 0']
    rooms_wo_outside = ['84, 139, 84', '0, 0, 0', '255, 0, 255', '115, 198, 205', '139, 26, 85', '130, 134, 139', '128, 0, 0', '255, 144, 30', '58, 58, 139', '235, 206, 135', '42, 42, 165', '0, 100, 0']
    # rooms = ['170,232,238', '128,128,240', '230,216,173', '0,215,255', '0,165,255', '35,142,107', '221,160,221', '0,255,255', '214,112,218']
    # IoUs, max_key = find_central_point(one_dim_gt_mask, one_dim_pred_mask)
    max_key = (0,0) # no shifting
    macro_iou, micro_iou = calculate_iou(rooms, dict_gt_mask, dict_pred_mask, max_key)
    
    Macro_IoUs.append(macro_iou)
    Micro_IoUs.append(micro_iou)

  1%|▏         | 156/12110 [07:23<9:32:49,  2.88s/it]

In [12]:
print(f"Macro IoU: {np.mean(Macro_IoUs)}")
print(f"Micro IoU: {np.mean(Micro_IoUs)}")

Macro IoU: 0.22276059528678854
Micro IoU: 0.19607894892981034
